In [2]:
!pip install tokenizers transformers

nltk.download("stopwords")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.9 MB/s eta 0:00:00


In [5]:
from tokenizers.processors import BertProcessing
from tokenizers import ByteLevelBPETokenizer
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    BertTokenizer,
    BertModel,
    AutoModelWithLMHead,
    AutoTokenizer
)
import nltk
from nltk.corpus import stopwords
# import pytorch_lightning as pl
import numpy as np
import argparse
import torch
from torch import nn
import torch.nn.functional as F
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Load pre-trained GPT2 language model weights
with torch.no_grad():
    gptmodel = GPT2LMHeadModel.from_pretrained('gpt2')
    gptmodel.eval()

# Load pre-trained GPT2 tokenizer
gpttokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# simple tokenizer + stemmer
regextokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
stemmer = nltk.stem.PorterStemmer()

In [30]:
max_score = 0.04883734556105129

In [31]:

def repetition_penalty(sentence):
    '''
    Adds a penalty for each repeated (stemmed) token in
    an utterance. Returns the total penalty of the sentence
    '''
    word_list = regextokenizer.tokenize(sentence.lower())
    filtered_words = [
        word for word in word_list if word not in stopwords.words('english')]
    stem_list = [stemmer.stem(word) for word in filtered_words]
    penalty = 0
    visited = []
    for w in stem_list:
        if w not in visited:
            visited.append(w)
        else:
            penalty += 0.001
    print(visited)
    print(penalty)
    return penalty

def perplexity(sentence):
    '''
    Computes the PPL of an utterance using GPT2 LM
    '''
    tokenize_input = gpttokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    with torch.no_grad():
        loss = gptmodel(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.detach().numpy())


def fluency_score(sentence):
    '''
    Computes the fluency score of an utterance, given by the
    inverse of the perplexity minus a penalty for repeated tokens
    '''
    ppl = perplexity(sentence)
    penalty = repetition_penalty(sentence)
    score = (1 / ppl) - penalty
    # normalise by the highest possible fluency computed on the corpus:
    normalized_score = score / max_score  # Normalize the score
    return normalized_score

    return score


In [32]:
import pandas as pd
data = pd.read_csv('new_utterances.csv')#put data in Drive root folder or change path
# data = data.drop(["fluency_score","sentiment_score","empathy_score"], axis=1)
data = data.dropna()
# data = data.drop(["Score"], axis=1)

In [33]:
data.head()

,sentences
0,Curiosity can help you fuel your natural drive...
1,Curiosity is like a spark that ignites your de...
2,Nurturing your curiosity can help you to keep ...
3,Cultivating your curiosity is essential for re...
4,Harnessing your curiosity is the key to stayin...


In [34]:
d1 = data['sentences'].tolist()
print(len(d1))

234


In [35]:
s1 = "Acquiring both characteristics of {a dichotomy} contributes to nurturing creativity, it could double your collection of responses and allow you to interact with a wider range of opportunities. It is like two yolks in one egg, twice the fun."
fluency_score(s1)

['acquir', 'characterist', 'dichotomi', 'contribut', 'nurtur', 'creativ', 'could', 'doubl', 'collect', 'respons', 'allow', 'interact', 'wider', 'rang', 'opportun', 'like', 'two', 'yolk', 'one', 'egg', 'twice', 'fun']
0


0.20334949790888182

In [38]:
data['fluency_score'] = ''


In [41]:
ls = []
for i, sentence in enumerate(d1):
    score = fluency_score(sentence)
    data.loc[i, 'fluency_score'] = score
    ls.append(score)

['curios', 'help', 'fuel', 'natur', 'drive', 'explor', 'discov', 'stay', 'open', 'follow', 'passion', 'curiou', 'import', 'becom', 'creativ', 'good', 'news', 'practic']
0.002
['curios', 'like', 'spark', 'ignit', 'desir', 'explor', 'discov', 'stay', 'open', 'new', 'experi', 'allow', 'chase', 'passion', 'embrac', 'key', 'unlock', 'creativ']
0.001
['nurtur', 'curios', 'help', 'keep', 'enthusiasm', 'explor', 'stay', 'open', 'mind', 'go', 'passion']
0
['cultiv', 'curios', 'essenti', 'remain', 'creativ', 'great', 'thing', 'someth', 'practic']
0
['har', 'curios', 'key', 'stay', 'excit', 'explor', 'keep', 'open', 'mind', 'pursu', 'passion']
0
['use', 'curios', 'fuel', 'investig', 'learn', 'stay', 'recept', 'chase', 'dream']
0.001
['capit', 'curios', 'help', 'maintain', 'zest', 'discoveri', 'stay', 'open', 'mind', 'follow', 'heart']
0
['develop', 'curios', 'essenti', 'stay', 'creativ', 'good', 'news', 'someth', 'hone']
0
['explor', 'curios', 'help', 'nurtur', 'natur', 'enthusiasm', 'stay', 'fle

In [42]:
print("mean: ", np.mean(ls))
print("std: " , np.std(ls))
print("max: ", max(ls))

mean:  0.2759056976451534
std:  0.1876623030670546
max:  1.0


In [43]:
data.to_csv('new_utterances_with_fluency_scores.csv', index=False)
